In [4]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow as tf


pre_trained_model = base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

    
pre_trained_model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 63, 63, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 63, 63, 32)   96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 63, 63, 32)   0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784
__________________________________________________________________________________________________


In [11]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


last layer output shape:  (None, 6, 6, 768)


In [12]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [13]:
batch_size = 10
img_height = 128
img_width = 128

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')
validation_datagen= ImageDataGenerator(rescale=1./255)

train_generator= train_datagen.flow_from_directory('dataset/train',
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='binary')
validation_generator= validation_datagen.flow_from_directory('dataset/validation',
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

Found 689 images belonging to 2 classes.
Found 687 images belonging to 2 classes.


In [14]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.99):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True
            

In [ ]:
callbacks=myCallback()
history=model.fit(train_generator,epochs=30,verbose=1,validation_data=validation_generator,callbacks=[callbacks])

Epoch 1/30
69/69 [==============================] - 26s 371ms/step - loss: 0.3946 - acc: 0.8505 - val_loss: 1.9449 - val_acc: 0.3828
Epoch 2/30
69/69 [==============================] - 22s 324ms/step - loss: 0.2389 - acc: 0.8955 - val_loss: 1.6781 - val_acc: 0.6492
Epoch 3/30
69/69 [==============================] - 22s 325ms/step - loss: 0.1687 - acc: 0.9390 - val_loss: 1.3198 - val_acc: 0.7525
Epoch 4/30
69/69 [==============================] - 23s 327ms/step - loss: 0.1447 - acc: 0.9492 - val_loss: 1.7373 - val_acc: 0.7380
Epoch 5/30
69/69 [==============================] - 23s 330ms/step - loss: 0.1478 - acc: 0.9536 - val_loss: 1.8385 - val_acc: 0.7409
Epoch 6/30
69/69 [==============================] - 23s 330ms/step - loss: 0.1679 - acc: 0.9463 - val_loss: 1.5750 - val_acc: 0.7802
Epoch 7/30
69/69 [==============================] - 23s 331ms/step - loss: 0.1405 - acc: 0.9550 - val_loss: 1.5648 - val_acc: 0.8006
Epoch 8/30
69/69 [==============================] - 23s 332ms/step - 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('saved_model/mask_inception_v3_dense1024.h5')